In [5]:
import pandas as pd
import matplotlib as plt
import sqlite3
import os

In [7]:
#retriing table names from data frame
im_path = r"C:\Users\HP\Desktop\moringa\PHASE1_PROJECT\im.db\im.db"

conn = sqlite3.connect(im_path)
cursor = conn.cursor()

table_name_query = """ SELECT name
                       AS 'Table Names' 
                       FROM sqlite_master 
                       WHERE type='table';
                       """

table_names = pd.read_sql(table_name_query, conn)

print(table_names)

     Table Names
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [9]:
q = """SELECT *
       FROM directors;
    """
df = pd.read_sql(q, conn)

print(df.head())

    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585
2  tt0835418  nm0151540
3  tt0835418  nm0151540
4  tt0878654  nm0089502


In [6]:
# marging the movie mudgets in csv file and  cleaning them up
budget_one = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\tn.movie_budgets.csv\\tn.movie_budgets.csv")
budget_two = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\bom.movie_gross.csv\\bom.movie_gross.csv")


In [15]:
budget_one.head()
budget_one.tail()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


In [39]:
print(budget_one.columns)

Index(['movie', 'domestic_gross', 'worldwide_gross'], dtype='object')


In [42]:
budget_one

,movie,domestic_gross,worldwide_gross
0,Avatar,"$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$620,181,382","$1,316,721,747"
...,...,...,...
5777,Red 11,$0,$0
5778,Following,"$48,482","$240,495"
5779,Return to the Land of Wonders,"$1,338","$1,338"
5780,A Plague So Pleasant,$0,$0


In [32]:
print(budget_two.columns)

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')


In [34]:
budget_two.drop(columns=["studio", "year"], inplace=True)
budget_two

,title,domestic_gross,foreign_gross
0,Toy Story 3,415000000.0,652000000
1,Alice in Wonderland (2010),334200000.0,691300000
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000
3,Inception,292600000.0,535700000
4,Shrek Forever After,238700000.0,513900000
...,...,...,...
3382,The Quake,6200.0,NaN
3383,Edward II (2018 re-release),4800.0,NaN
3384,El Pacto,2500.0,NaN
3385,The Swan,2400.0,NaN


In [54]:
budget_two.columns = ['movie', 'domestic_gross', 'worldwide_gross']

# Print the updated DataFrame
print(budget_two)

                                            movie  domestic_gross  \
0                                     Toy Story 3     415000000.0   
1                      Alice in Wonderland (2010)     334200000.0   
2     Harry Potter and the Deathly Hallows Part 1     296000000.0   
3                                       Inception     292600000.0   
4                             Shrek Forever After     238700000.0   
...                                           ...             ...   
3382                                    The Quake          6200.0   
3383                  Edward II (2018 re-release)          4800.0   
3384                                     El Pacto          2500.0   
3385                                     The Swan          2400.0   
3386                            An Actor Prepares          1700.0   

     worldwide_gross  
0          652000000  
1          691300000  
2          664300000  
3          535700000  
4          513900000  
...              ...  
3382      

In [56]:
merged_revenue=pd.concat([budget_one, budget_two])
merged_revenue

,movie,domestic_gross,worldwide_gross
0,Avatar,"$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$620,181,382","$1,316,721,747"
...,...,...,...
3382,The Quake,6200.0,NaN
3383,Edward II (2018 re-release),4800.0,NaN
3384,El Pacto,2500.0,NaN
3385,The Swan,2400.0,NaN


In [57]:
merged_revenue.drop_duplicates()

,movie,domestic_gross,worldwide_gross
0,Avatar,"$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$241,063,875","$1,045,663,875"
2,Dark Phoenix,"$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,"$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$620,181,382","$1,316,721,747"
...,...,...,...
3382,The Quake,6200.0,NaN
3383,Edward II (2018 re-release),4800.0,NaN
3384,El Pacto,2500.0,NaN
3385,The Swan,2400.0,NaN


In [70]:
merged_revenue['domestic_gross'].mean()

11417962.000218127

In [71]:
median_value=merged_revenue['domestic_gross'].median()

In [72]:
merged_revenue['domestic_gross'].max()

936700000.0

In [73]:
merged_revenue['domestic_gross'].describe()

count    9.169000e+03
mean     1.141796e+07
std      4.262574e+07
min      1.000000e+02
25%      1.400000e+06
50%      1.400000e+06
75%      1.400000e+06
max      9.367000e+08
Name: domestic_gross, dtype: float64

In [74]:
merged_revenue['domestic_gross'] = merged_revenue['domestic_gross'].fillna(median_value)

In [ ]:
merged_revenue['worldwide_gross'].describe()

In [78]:
# im trying to remove the /$ signs 
merged_revenue['worldwide_gross'] = pd.to_numeric(merged_revenue['worldwide_gross'].replace('[\$,]', '', regex=True), errors='coerce')


In [80]:
median=merged_revenue['worldwide_gross'].median()

In [82]:
merged_revenue['worldwide_gross'] = merged_revenue['worldwide_gross'].fillna(median)

In [ ]:
merged_revenue['worldwide_gross'].describe()